# Finne emneansvarlege
Eg prøver å finne alle emneansvarlege ved HVL i eit gitt semester; dette kan vi bruke til å "spisse" informasjon (etter ønske frå Alisa).

In [1]:
import requests
import os
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta, date

FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

hodeCanvas = {
    'Authorization': f'Bearer {os.environ["tokenCanvas"]}',
}

parametre = {'per_page': 100}

today = date.today()
idag = today.isoformat()
igår = (today - timedelta(days=1)).isoformat()


In [16]:
timestamp ="2025-03-14T02:16:19Z"
dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
dt.hour 

2

In [18]:
def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return {}

## Sjekk kva som vert overført frå FS til Canvas
Eg kan sjekke kva emne som vert overført kvar natt, for det må vel vere "aktive" emne? Eitt spørsmål då er jo *når* i terminen eg skal ta ut ei slik liste, men det får eg sjå på. 

Først henter eg ut alle emne som vert overført frå FS til Canvas. Det gjer eg ved å hente alle sis_import-filene, og så finne den som vart starta av systemet før 0300 om natta (i dag), og som har eit vedlegg som heiter "courses.csv". Når alle desse tre er oppfylt henter eg ein URL til ei fil, og les inn data frå den fila:

In [17]:
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
respons = requests.get(url, headers=hodeCanvas, params=parametre).json()
for r in respons['sis_imports']:
    if (r['user']['id']==1596) and (datetime.strptime(r['created_at'], "%Y-%m-%dT%H:%M:%SZ").hour < 3):
        for v in r['csv_attachments']:
            if v['filename'] == "courses.csv":
                fil = v['url']
                data = pd.read_csv(StringIO(requests.get(fil, headers=hodeCanvas).content.decode("utf-8")))
                break
        break



In [19]:
emne = pd.read_csv(StringIO(requests.get(fil, headers=hodeCanvas).content.decode("utf-8")))


In [ ]:
emne = pd.read_csv("courses.csv")

In [5]:
emne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11362 entries, 0 to 11361
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            11362 non-null  object 
 1   short_name           11362 non-null  object 
 2   long_name            11362 non-null  object 
 3   account_id           11362 non-null  object 
 4   term_id              8008 non-null   object 
 5   status               11362 non-null  object 
 6   start_date           0 non-null      float64
 7   end_date             0 non-null      float64
 8   blueprint_course_id  0 non-null      float64
dtypes: float64(3), object(6)
memory usage: 799.0+ KB


Så tar eg vekk alle emne som ikkje har termin angitt eller som starter på "KK" (det er ikkje ordinære undervisingemne):

In [6]:
temp = emne[~emne['course_id'].str.startswith('KK')]
emne_no = temp[~temp['term_id'].isna()]

Så finn eg alle emne som har undervising "2025_VÅR"; det er alle der startsemester er mindre enn eller lik "2025_VÅR" og sluttsemester er større enn eller lik "2025_VÅR". Først lager eg eit "desimalsemester" som er slutter på 0.0 i vårsemesteret og 0.5 i haustsemesteret, og så plukker eg ut alle som har "2025" i intervallet:

In [7]:
def sjekk_aktiv(rekke):
    sem1 = rekke['semester1']
    sem2 = rekke['semester2']
    return (sem1 <= 2025 and sem2 >= 2025)

emne_no['startår'] = emne_no['term_id'].apply(lambda x: x.split('-')[0].split('_')[0])
emne_no['sluttår'] = emne_no['term_id'].apply(lambda x: x.split('-')[1].split('_')[0] if '-' in x else x.split('_')[0])
emne_no['starttermin'] = emne_no['term_id'].apply(lambda x: x.split('-')[0].split('_')[1])
emne_no['slutttermin'] = emne_no['term_id'].apply(lambda x: x.split('-')[1].split('_')[1] if '-' in x else x.split('_')[0])
emne_no['semester1'] = emne_no['startår'].apply(lambda x:int(x)) + 0.5*(emne_no['starttermin']=="HØST")
emne_no['semester2'] = emne_no['sluttår'].apply(lambda x:int(x)) + 0.5*(emne_no['slutttermin']=="HØST")
emne_no['aktiv'] = emne_no.apply(sjekk_aktiv, axis=1)
aktive_emne = emne_no[emne_no['aktiv']]

C:\Users\AKV\AppData\Local\Temp\ipykernel_20280\1422657581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emne_no['startår'] = emne_no['term_id'].apply(lambda x: x.split('-')[0].split('_')[0])
C:\Users\AKV\AppData\Local\Temp\ipykernel_20280\1422657581.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emne_no['sluttår'] = emne_no['term_id'].apply(lambda x: x.split('-')[1].split('_')[0] if '-' in x else x.split('_')[0])
C:\Users\AKV\AppData\Local\Temp\ipykernel_20280\1422657581.py:8: SettingWithCopyWar

Til slutt plukker eg ut alle emnekodene frå feltet `course_id` og finn alle unike koder:

In [8]:
def finn_emnekode(streng):
    if streng.startswith("UA") or streng.startswith("UE"):
        return streng.split('_')[2]
    else:
        return None
aktive_emne['emnekode'] = aktive_emne['course_id'].apply(finn_emnekode)
aktive_emne['emnekode'] = aktive_emne['course_id'].apply(finn_emnekode)
unike_emne = aktive_emne['emnekode'].unique()

C:\Users\AKV\AppData\Local\Temp\ipykernel_20280\47494943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aktive_emne['emnekode'] = aktive_emne['course_id'].apply(finn_emnekode)
C:\Users\AKV\AppData\Local\Temp\ipykernel_20280\47494943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aktive_emne['emnekode'] = aktive_emne['course_id'].apply(finn_emnekode)


Så kan eg leite opp emneansvarleg for kvart emnne (med GraphQL mot FS):

In [9]:
query = """
query Emneansvarleg($emnekode: [String!], $arstall: Int!, $terminbetegnelse: EmneIkkeUtloptITerminTerminbetegnelse!) {
    emner(
    filter: {eierInstitusjonsnummer: "203", emnekoder: $emnekode, ikkeUtloptITermin: {arstall: $arstall, terminbetegnelse: $terminbetegnelse}}
  ) {
    nodes {
      personroller {
        aktiv
        fagperson {
          navn {
            etternavn
            fornavn
          }
          feideBruker
        }
      }
      organisasjonsenhet {
        studieAnsvarlig {
          instituttnummer
          fakultet {
            fakultetsnummer
          }
        }
      }
    }
  }
}
"""

In [10]:
dataliste = []
for e in unike_emne:
    variable = {'emnekode': e, 'arstall': 2024, 'terminbetegnelse': 'VAR'}
    svar = query_FS_graphql(query, variable)
    data = svar['data']['emner']['nodes']
    for d in data:
        fakultet = d['organisasjonsenhet']['studieAnsvarlig']['fakultet']['fakultetsnummer']
        institutt = d['organisasjonsenhet']['studieAnsvarlig']['instituttnummer']
        for p in d['personroller']:
            if p['aktiv']:
                try:
                    etternamn = p['fagperson']['navn']['etternavn']
                    fornamn = p['fagperson']['navn']['fornavn']
                    epost = p['fagperson']['feideBruker']
                    dataliste.append([fakultet, institutt,e, etternamn, fornamn, epost])

                    print(fakultet, institutt, e, etternamn, fornamn, epost)
                except:
                    pass


30 2 BULLA100 Hjelle Kristine krhj@hvl.no
30 3 STMA100 Hansen Hannah Belsvik hhh@hvl.no
30 0 ABLUPR201 Kvarme Henriette Opsanger heok@hvl.no
30 0 ABLUPR401 Kvarme Henriette Opsanger heok@hvl.no
20 10 ADA502 Kristensen Lars Michael lmkr@hvl.no
20 10 ADA502 Stünkel Patrick past@hvl.no
20 10 ADA502 Oyetoyan Tosin Daniel tdoy@hvl.no
20 10 ADA504 Rutle Adrian aru@hvl.no
20 10 ADA504 Lamo Yngve yla@hvl.no
20 10 ADA521 Kyrkjebø Erik erikky@hvl.no
20 10 ADA521 Kristensen Lars Michael lmkr@hvl.no
20 10 ADA522 Kyrkjebø Erik erikky@hvl.no
20 10 ADA522 Kristensen Lars Michael lmkr@hvl.no
20 10 ADA523 Kyrkjebø Erik erikky@hvl.no
20 10 ADA523 Kristensen Lars Michael lmkr@hvl.no
20 10 ADA524 Kyrkjebø Erik erikky@hvl.no
20 10 ADA524 Kristensen Lars Michael lmkr@hvl.no
20 10 ADA527 Kyrkjebø Erik erikky@hvl.no
20 10 ADA528 Kampen Anne-Lena alk@hvl.no
20 10 ADA528 Fojcik Marcin Andrzej marcinf@hvl.no
20 10 ADA529 Rahman Talal tra@hvl.no
20 10 ADA529 Serea Oana Silvia oss@hvl.no
20 10 ADA595 Kyrkjebø Erik

In [11]:
emneansvarlege = pd.DataFrame(dataliste)
emneansvarlege.to_csv("emneansvarlege.csv", index=False)

## Emne per fakultet/institutt
Kan eg også fordele emna på fakultet/institutt? Eg har endra skriptet slik at det ber om nummer på fakultet og institutt, og så samler eg alle i eit passe rekneark. Så kan ein filtrere ut dei ein vil ha tak i.

In [ ]:
emneansvarlege = pd.DataFrame(dataliste, columns = ['fakultet', 'institutt','emnekode', 'etternamn', 'fornamn', 'epost'])
emneansvarlege.to_excel("emneansvarlege.xlsx", index=None)